In [1]:
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision.utils as vutils
import random
import os
import shutil
import torch
import torch.nn as nn
from torch import optim
from torch.autograd import Variable
import numpy as np
import os

In [5]:
'''
Set Defaults
'''
num_epochs = 200 
batch_size = 64
num_classes = 10 # number of classes for dataset
lr = 0.0002 
b1 = 0.5 # adam: decay of first order momentum of gradient
b2 = 0.999 # adam: decay of first order momentum of gradient
n_cpu = 8 # number of cpu threads to use during batch generation
latent_dim = 100 # dimensionality of the latent space
img_size = 32 # size of each image dimension
channels = 1 # number of output image channels
sample_interval = 400 # interval between image sampling
ndf = 16

In [3]:
# Set cuda
if torch.cuda.is_available():
    cuda = True 
else:
    cuda = False

In [4]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.zero_()

In [ ]:
class Net(nn.Module):
    def __init__(self, ndf, alpha, nc, drop_rate, num_classes):
        super(net, self).__init__()
        self.use_gpu = cuda
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=nc, out_channels=ndf, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf),
            nn.LeakyReLU(alpha),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=ndf, out_channels=ndf*2, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(alpha),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=ndf*2, out_channels=ndf*4, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf*4),
            nn.LeakyReLU(alpha),
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=ndf*4, out_channels=ndf*8, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf*8),
            nn.LeakyReLU(alpha),
        )
        
        self.main = nn.Sequential(
            
            # input is (number_channels) x 32 x 32
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(alpha),
            nn.Dropout2d(drop_rate),
            # (ndf) x 16 x 16
            nn.Conv2d(ndf, ndf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf),
            nn.LeakyReLU(alpha),
            # (ndf) x 8 x 8
            nn.Conv2d(ndf, ndf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf),
            nn.LeakyReLU(alpha),
            nn.Dropout2d(drop_rate),
            # (ndf) x 4 x 4
            nn.Conv2d(ndf, ndf * 2, 3, 1, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(alpha),
            # (ndf * 2) x 4 x 4
            nn.Conv2d(ndf * 2, ndf * 2, 3, 1, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(alpha),
            # (ndf * 2) x 4 x 4
            nn.Conv2d(ndf * 2, ndf * 2, 3, 1, 0, bias=False),
            nn.LeakyReLU(alpha),
            # (ndf * 2) x 2 x 2
        )
        
        self.features = nn.AvgPool2d(kernel_size=2)

        self.class_logits = nn.Linear(
            in_features=(ndf * 2) * 1 * 1,
            out_features=num_classes + 1)
        
        self.gan_logits = _ganLogits(num_classes)
        
        self.softmax = nn.Softmax(dim=0)

    def forward(self, inputs):
#         if isinstance(inputs.data, torch.cuda.FloatTensor) and self.use_gpu:
#             out = nn.parallel.data_parallel(self.main, inputs, range(1))
#         else:
        out = self.main(inputs)

        features = self.features(out)
        print("features before = ", features.size())
        features = features.squeeze()
        print("features after = ", features.size())

        class_logits = self.class_logits(features)
        print("class logits = ", class_logits.size())

        gan_logits = self.gan_logits(class_logits)
        print("gan_logits = ", gan_logits.size())
        
        out = self.softmax(class_logits)

        return out, class_logits, gan_logits, features